In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Check if GPU is available and set memory limit
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8192)])  # Limit memory to 8GB
    except RuntimeError as e:
        print(e)

# Set parameters
image_size = (224, 224)  # Standard size for the input layer
batch_size = 16
epochs = 50  # Number of epochs for training
repeat_count = 3  # Number of times to repeat the training process

# Augmentation and Preprocessing for focusing on color and texture
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Normalizing
    zoom_range=0.2,            # Slight zoom
    rotation_range=20,         # Rotations
    width_shift_range=0.2,     # Shifting the image horizontally
    height_shift_range=0.2,    # Shifting the image vertically
    horizontal_flip=True,      # Horizontal flipping
    vertical_flip=True,        # Vertical flipping
    brightness_range=[0.8, 1.2] # Varying brightness for bruises
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Automatically detect classes in alphabetical order
train_data_dir = r"C:\Users\ritish-flipkart\Downloads\flipkart\flipkart\dataset_sorted\train"
class_names = sorted(os.listdir(train_data_dir))
num_classes = len(class_names)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    r"C:\Users\ritish-flipkart\Downloads\flipkart\flipkart\dataset_sorted\train",  # Updated training dataset path
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    classes=class_names  # Ensure classes are in alphabetical order
)

# Validation data generator
validation_generator = validation_datagen.flow_from_directory(
    r"C:\Users\ritish-flipkart\Downloads\flipkart\flipkart\dataset_sorted\validation",  # Updated validation dataset path
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    classes=class_names  # Ensure classes are in alphabetical order
)

# Initialize the model
model = Sequential()

# Convolutional Block 1: Extracting color features
model.add(Input(shape=(224, 224, 3)))  # Input layer
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())  # Normalization after Conv layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Block 2: Enhancing color and texture features
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Block 3: Further extracting texture features
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Block 4: Focusing on regions of interest (bruises)
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Block 5: Fine-tuning to detect subtle textures and colors (like bruises)
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening and fully connected layers for classification
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Regularization to prevent overfitting
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Regularization to prevent overfitting
model.add(Dense(num_classes, activation='softmax'))  # Output layer for classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks for early stopping and saving the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_fruit_model.keras', monitor='val_loss', save_best_only=True)

# Repeat the training process for a specified number of times
for i in range(repeat_count):
    print(f"Starting training iteration {i + 1}/{repeat_count}")
    print("Class names in alphabetical order:", class_names)  # Print class names before each iteration
    # Train the model and store the training history
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks=[early_stopping, model_checkpoint]
    )

    # Plot training and validation accuracy and loss using matplotlib
    plt.figure(figsize=(12, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Training and Validation Accuracy (Iteration {i + 1})')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss (Iteration {i + 1})')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()
